In [ ]:
import pandas as pd


def loadPrices():
    fn="../prices.txt"
    global nt, nInst
    df=pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    return (df.values).T

prices = loadPrices()


[[38.25 38.11 37.9  ... 36.58 36.43 36.24]
 [69.06 69.98 69.56 ... 50.8  51.19 50.64]
 [62.08 62.39 62.82 ... 44.25 44.02 43.43]
 ...
 [24.02 24.04 23.76 ... 15.81 15.75 15.74]
 [76.03 76.05 75.66 ... 80.93 80.8  80.5 ]
 [27.5  27.54 27.55 ... 31.83 31.78 31.59]]
